# 1️⃣ Concepto de Stacking con Redes Neuronales (NN)

**Stacking:**

- Entrenas varias redes base \(M_1, M_2, M_3, \dots\) sobre tu dataset.
- Cada red produce una predicción \(y_i\) (por ejemplo, **logits** o **probabilidades**).
- Estas predicciones se usan como **features** para un modelo final llamado **meta-modelo**.
- El **meta-modelo** aprende los **pesos óptimos** para combinar las salidas de las redes base.

📌 **Ventaja:**

- Permite que la combinación final sea **no lineal**, mucho más flexible que un simple promedio o votación.
- Se puede usar otra **red neuronal pequeña** como meta-modelo.


In [ ]:
# 📘 Notebook: Stacking MLPs optimizado para MNIST

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

# ================================================
# 1️⃣ Preparación de datos MNIST
# ================================================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Para demo, usamos un subset pequeño (puedes usar el dataset completo)
train_dataset = Subset(train_dataset, range(20000))  
test_dataset = Subset(test_dataset, range(5000))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

device = torch.device("cpu")
print("Usando:", device)

# ================================================
# 2️⃣ Definición de redes base
# ================================================
class BaseNN(nn.Module):
    def __init__(self, input_size=784, hidden_size=256, output_size=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, output_size)
        )
        
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten 28x28 → 784
        return self.net(x)

# ================================================
# 3️⃣ Entrenamiento de cada base y evaluación
# ================================================
def train_base(model, epochs=10):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            optimizer.step()
            total_loss += loss.item()
    
    # Accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            correct += (y_pred.argmax(dim=1) == y).sum().item()
            total += y.size(0)
    acc = correct / total
    return acc

# Crear y entrenar 3 redes base
base_models = [BaseNN().to(device) for _ in range(3)]
base_accs = []

for i, model in enumerate(base_models):
    print(f"Entrenando BaseNN {i+1}...")
    acc = train_base(model)
    base_accs.append(acc)
    print(f"Accuracy BaseNN {i+1}: {acc:.4f}")

# ================================================
# 4️⃣ Construcción de dataset para meta-modelo
# ================================================


# =========================================================
# Función: get_logits
# =========================================================
# Esta función tiene como objetivo calcular las salidas "logits" de un conjunto 
# de modelos base sobre un conjunto de datos dado. Es fundamental en técnicas 
# de ensemble tipo "stacking", donde usamos las predicciones de modelos base 
# como características de entrada para un meta-modelo.
#
# Conceptos clave:
# 1️⃣ Logits:
#    - Los logits son las salidas directas de la última capa lineal de una red 
#      neuronal antes de aplicar cualquier función de activación como softmax. 
#    - Por ejemplo, en clasificación de 10 clases (MNIST), un modelo devuelve un 
#      vector de 10 números reales por cada muestra. Cada número representa 
#      la "evidencia" o "puntaje sin normalizar" para esa clase.
#    - El meta-modelo generalmente aprende a combinar estas evidencias de varios 
#      modelos base para mejorar la predicción final.
#
# 2️⃣ detach():
#    - `.detach()` crea un nuevo tensor que comparte los datos con el original, 
#      pero **corta el grafo de autograd**. Es decir, PyTorch ya no rastreará 
#      operaciones posteriores para calcular gradientes respecto a este tensor.
#    - Esto es crítico aquí porque:
#      * No queremos que el cálculo de la pérdida del meta-modelo retropropague 
#        gradientes hacia los modelos base, ya que esos modelos ya están entrenados.
#      * Si no usamos detach(), PyTorch intentaría construir un grafo enorme 
#        combinando el meta-modelo y los modelos base, aumentando memoria y riesgo 
#        de errores de gradientes.
#      * Además, se gastarían recursos innecesarios recalculando gradientes que 
#        no necesitamos, ralentizando el entrenamiento.
#
# Flujo detallado de la función:
# -----------------------------------------
# 1. Inicializa dos listas vacías:
#    - all_logits → para almacenar los logits concatenados de todos los modelos
#    - all_labels → para almacenar las etiquetas verdaderas correspondientes
#
# 2. Itera por cada batch del DataLoader:
#    a) x, y = batch
#       - x: features de entrada del batch
#       - y: etiquetas verdaderas del batch
#    b) x = x.to(device)
#       - Mueve los datos al mismo dispositivo que los modelos (CPU o GPU)
#    c) logits = [m(x).detach() for m in models]
#       - Calcula la salida de cada modelo base sobre x
#       - Se usa detach() para romper la conexión con los grafos de los modelos base
#       - Esto devuelve los logits como tensores que **no participan en retropropagación**
#    d) stacked_logits = torch.cat(logits, dim=1)
#       - Concatenamos horizontalmente los logits de todos los modelos
#       - Si hay 3 modelos y cada uno devuelve 10 logits, stacked_logits tendrá 30 
#         columnas por muestra
#    e) all_logits.append(stacked_logits.cpu())
#       - Movemos los logits a CPU para reducir uso de GPU y los almacenamos
#    f) all_labels.append(y)
#       - Guardamos las etiquetas originales para entrenar/evaluar el meta-modelo
#
# 3. Al final, usamos torch.cat() sobre los batches para devolver un tensor completo:
#    - all_logits → tensor de tamaño (num_muestras, suma_de_logits_por_modelo)
#    - all_labels → tensor de tamaño (num_muestras,)
#
# Por qué es útil y cómo se implementa en stacking:
# -----------------------------------------
# - Cada modelo base puede aprender patrones distintos en los datos.
# - El meta-modelo toma como entrada todos los logits y aprende a ponderarlos 
#   para mejorar la predicción final.
# - Gracias a detach(), los pesos de los modelos base permanecen fijos; sólo se 
#   entrena el meta-modelo.
# - Esta implementación es eficiente: procesamos batches completos, movemos 
#   datos a CPU para almacenamiento y evitamos construir grafos innecesarios.
#
# Ejemplo de uso:
# meta_X_train, meta_y_train = get_logits(base_models, train_loader)
# meta_X_test, meta_y_test = get_logits(base_models, test_loader)
# - meta_X_train / meta_X_test → entrada para el meta-modelo
# - meta_y_train / meta_y_test → etiquetas verdaderas para entrenamiento/evaluación


def get_logits(models, loader):
    all_logits = []
    all_labels = []
    for x, y in loader:
        x = x.to(device)
        logits = [m(x).detach() for m in models]  # <-- Aquí usamos detach()
        stacked_logits = torch.cat(logits, dim=1)
        all_logits.append(stacked_logits.cpu())
        all_labels.append(y)
    return torch.cat(all_logits), torch.cat(all_labels)

meta_X_train, meta_y_train = get_logits(base_models, train_loader)
meta_X_test, meta_y_test = get_logits(base_models, test_loader)

# ================================================
# 5️⃣ Definición y entrenamiento del meta-modelo
# ================================================
class MetaNN(nn.Module):
    def __init__(self, input_size=30, hidden_size=128, output_size=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
        
    def forward(self, x):
        return self.net(x)

meta_model = MetaNN().to(device)
optimizer = optim.Adam(meta_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Dataset PyTorch para meta-modelo
meta_train_loader = DataLoader(list(zip(meta_X_train, meta_y_train)), batch_size=128, shuffle=True)
meta_test_loader = DataLoader(list(zip(meta_X_test, meta_y_test)), batch_size=128, shuffle=False)

# Entrenamiento
for epoch in range(10):
    meta_model.train()
    total_loss = 0
    for x, y in meta_train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = meta_model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        nn.utils.clip_grad_norm_(meta_model.parameters(), max_norm=2.0)
        optimizer.step()
        total_loss += loss.item()
    print(f"Meta-model Epoch {epoch+1} | Loss: {total_loss/len(meta_train_loader):.4f}")

# Evaluación meta-modelo
meta_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x, y in meta_test_loader:
        x, y = x.to(device), y.to(device)
        y_pred = meta_model(x)
        correct += (y_pred.argmax(dim=1) == y).sum().item()
        total += y.size(0)
meta_acc = correct / total

print("\n✅ Accuracy final del meta-modelo:", round(meta_acc,4))
print("Accuracy de cada BaseNN:", [round(a,4) for a in base_accs])


Usando: cpu
Entrenando BaseNN 1...
Accuracy BaseNN 1: 0.9610
Entrenando BaseNN 2...
Accuracy BaseNN 2: 0.9602
Entrenando BaseNN 3...
Accuracy BaseNN 3: 0.9614
Meta-model Epoch 1 | Loss: 0.1359
Meta-model Epoch 2 | Loss: 0.0109
Meta-model Epoch 3 | Loss: 0.0093
Meta-model Epoch 4 | Loss: 0.0071
Meta-model Epoch 5 | Loss: 0.0058
Meta-model Epoch 6 | Loss: 0.0061
Meta-model Epoch 7 | Loss: 0.0059
Meta-model Epoch 8 | Loss: 0.0054
Meta-model Epoch 9 | Loss: 0.0059
Meta-model Epoch 10 | Loss: 0.0044

✅ Accuracy final del meta-modelo: 0.964
Accuracy de cada BaseNN: [0.961, 0.9602, 0.9614]
